# Default Perturbation network analysis notebook
This notebook was automatically generated using freenrgworkflows   
Author: Antonia Mey   
Email: antonia.mey@ed.ac.uk

In [ ]:
%pylab inline
import networkanalysis.networkanalysis as n_graph
import networkanalysis.plotting as n_plot
import networkanalysis.experiments as n_ex
import networkanalysis.stats as n_stats
import networkanalysis
networkanalysis.__version__

In [ ]:
# Creating and populating the perturbation network
pG = n_graph.PerturbationGraph()
# Change the path below to the csv file containing the individual perturbations
pG.populate_pert_graph('/Users/toni_brain/Projects/git/freenrgworkflows/tests/io/graph.csv')
# Uncomment below if you have run multiple runs for some perturbations and add file path
#pG.add_data_to_graph('/path/to/additional/runs.csv')
target_compound = pG.compoundList[0] #change this to your target comound
pG.compute_weighted_avg_paths(target_compound)
pG.format_free_energies(merge_BM=True,intermed_ID='INT')
computed_relative_DDGs = pG.freeEnergyInKcal
print ("Free energies computed from the perturbation network are: ")
print ("---------------------------------------- ")
pG.write_free_energies(computed_relative_DDGs)


### Experimental data
It is useful to compare computed free energies to experimental data.
The cells below will read in your experimental data. Just replace the path to you IC50 data in the
`IC_50_file` variable 

In [ ]:
experiments = n_ex.ExperimentalData()
IC_50_file = 'tests/io/ic50_exp.dat'
experiments.compute_DDG_from_IC50s(IC_50_file, reference=target_compound)
experimental_DDGs = experiments.freeEnergiesInKcal
print ("Free energies computed from IC50 data: ")
print ("---------------------------------------- ")
pG.write_free_energies(experimental_DDGs)


### Typical plots
Below a bar plot and scatter plot template for comparing experimental and computed free energy values

In [ ]:
plotter = n_plot.FreeEnergyPlotter(experimental_DDGs, computed_relative_DDGs)
ax,fig = plotter.plot_bar_plot(legend=('experimental', 'computed'))

In [ ]:
plotter.plot_scatter_plot() 


### Error analysis on typical statistical measures: R, MUE and Kendall tau
Below are examples of how to resample from the data in order to obtain errorbars on Correlation coefficients, 
mean unsigned errors and Kendall tau. Returned are confidence interavals of 1 sigma. However standard deviations can also 
be returned. 

In [ ]:
stats = n_stats.freeEnergyStats()
stats.generate_statistics(computed_relative_DDGs,experimental_DDGs,repeats=10000)
r_error = stats.R_error
tau_error = stats.tau_error
mue_error = stats.mue_error
print ("R is: %.2f < %.2f < %.2f" %(r_error[0], stats.R, r_error[1]))
print ("Mue is: %.2f < %.2f < %.2f" %(mue_error[0], stats.mue, mue_error[1]))
print ("tau is: %.2f < %.2f < %.2f" %(tau_error[0], stats.tau, tau_error[1]))